# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [91]:
!py3 -m spacy download en_core_web_sm

Der Befehl "py3" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [93]:
!pip install -U spacy
!pip install -U typing_extensions
!python -m spacy download en_core_web_sm

Requirement already up-to-date: spacy in c:\users\ajl\anaconda3\lib\site-packages (3.7.1)
  Using cached https://files.pythonhosted.org/packages/ec/6b/63cc3df74987c36fe26157ee12e09e8f9db4de771e0f3404263117e75b95/typing_extensions-4.7.1-py3-none-any.whl
  Found existing installation: typing-extensions 4.4.0
    Uninstalling typing-extensions-4.4.0:
      Successfully uninstalled typing-extensions-4.4.0


ERROR: wasabi 1.1.2 has requirement typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you'll have typing-extensions 4.7.1 which is incompatible.
ERROR: thinc 8.2.1 has requirement typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you'll have typing-extensions 4.7.1 which is incompatible.
ERROR: spacy 3.7.1 has requirement typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you'll have typing-extensions 4.7.1 which is incompatible.
ERROR: confection 0.1.3 has requirement typing-extensions<4.5.0,>=3.7.4.1; python_version < "3.8", but you'll have typing-extensions 4.7.1 which is incompatible.


  Using cached https://files.pythonhosted.org/packages/0b/8e/f1a0a5a76cfef77e1eb6004cb49e5f8d72634da638420b9ea492ce8305e8/typing_extensions-4.4.0-py3-none-any.whl
  Found existing installation: typing-extensions 4.7.1
    Uninstalling typing-extensions-4.7.1:
      Successfully uninstalled typing-extensions-4.7.1
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


ERROR: pydantic 2.4.2 has requirement typing-extensions>=4.6.1, but you'll have typing-extensions 4.4.0 which is incompatible.
ERROR: pydantic-core 2.10.1 has requirement typing-extensions!=4.7.0,>=4.6.0, but you'll have typing-extensions 4.4.0 which is incompatible.


In [84]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AJL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\AJL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\AJL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\AJL\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


ModuleNotFoundError: No module named 'spacy'

In [152]:
# load data from database
engine = create_engine('sqlite:///../data/message_data.db')
df = pd.read_sql_table('Messages', engine).iloc[:3000]


In [153]:
X = df['message']
Y = df.iloc[:, 5:7]

In [154]:
import spacy
from spacy import displacy
from collections import Counter
nlp = spacy.load("en_core_web_sm")

In [155]:
X.iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

0.045454545454545456

In [156]:
def pct_capital_letters(t):
    return len(re.findall("[A-Z]", t))/len(t)

def question_mark(t):
    return '?' in t

def exclamation_mark(t):
    return '!' in t

In [157]:
def pct_word_types(t):
    t = t.strip()
    if len(t) == 0:
        return pd.Series()
    text = word_tokenize(t)
    ser =  pd.DataFrame(nltk.pos_tag(text, tagset='universal'))[1].value_counts()
    return ser/ser.sum()

### 2. Write a tokenization function to process your text data

In [158]:
def tokenize(text, replace_named_entities=False):
    
    if replace_named_entities:
        doc = nlp(t)
        for ele in doc.ents:
            t = t.replace(ele.text, ele.label_)
    
    text = text.lower()
    text = re.sub("[^a-z0-9]", " ", text)
    words = word_tokenize(text) 
    words = filter(lambda x: not(x in stopwords.words("english")), words)
    

    #Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    return stemmed


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [159]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [160]:
cv = CountVectorizer(tokenizer=tokenize)
tf = TfidfTransformer()
cl = MultiOutputClassifier(RandomForestClassifier(), n_jobs=8)


pipeline = Pipeline([
    ('preprocess', Pipeline([
        ('count', cv),
        ('tfid', tf)
    ])),
    ('classifier', cl)
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [162]:
# ideas, date, places, numbers, urls

In [163]:
model = pipeline.fit(X_train, y_train)

In [164]:
y_pred = pd.DataFrame(model.predict(X_test), columns=y_test.columns)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [166]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score

In [167]:
classification = dict()
for c in y_pred.columns:
    comp = (y_test[c], y_pred[c])
    classification[(c, 'precision')] = precision_score(*comp)
    classification[(c, 'recall')] = recall_score(*comp)
    classification[(c, 'f1')] = f1_score(*comp)


C:\Users\AJL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AJL\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [168]:
classification = pd.Series(classification).unstack()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.